In [2]:
import numpy
import h5py

In [4]:
with h5py.File('/home/ecarroll/PhD/2D_Navier_Stokes/Data/Tmp/SIM_DATA_NAVIER_RK4_FULL_N[512,512]_T[0-1]_NU[0.003000]_CFL[0.70]_u0[MAX_PALIN]_TAG[Sync-Test]/Main_HDF_Data.h5', 'r') as f:
    solv = f['Iter_00000']['w']

In [7]:
with h5py.File('/home/ecarroll/PhD/2D_Navier_Stokes/Data/InitialConditions/MaxPalinstrophy/maxdpdt_P10000_N512_IG0_psi_f.h5', 'r') as f:
    solv = f['psi']